In [33]:
from TM1py import TM1Service,Rules
import logging

In [226]:
#获取列表，并将列表的文本组合成长字符串用换行符链接
def handle_rule_list(li:list):
    s=''
    cnt=1
    n=len(li)
    for l in li:
        if cnt==1:
            s=l
        elif cnt==n:
            s=s+';'+'\n'+l+';'
        else:
            s=s+';'+'\n'+l
        cnt=cnt+1
    return s

In [495]:
#获取某维度中某个节点下全部的项目
def descendants(tm1:TM1Service,dim:str,element:str):
    hier=tm1.hierarchies.get(dimension_name=dim,hierarchy_name=dim)
    desc=hier.get_descendants(element_name=element,recursive=True)
    ele=tm1.elements.get(dim,dim,element)
    desc=list(desc)
    desc.append(ele)
    return desc

In [494]:
#获取某cube中的rules,忽略skipcheck&feeders特定开头结尾，并处理成列表
def rules(tm1:TM1Service,cube:str):
    cube=tm1.cubes.get(cube)
    rules=cube.rules
    if rules is None:
        rule_list=[]
    else:   
        rule_list=rules.rule_statements
        #忽略skipcheck
        if str.lower(rule_list[0])=='skipcheck':
            rule_list.pop(0)
        #忽略feeders
        if str.lower(rule_list[-1])=='feeders':
            rule_list.pop(-1)
    return rule_list

In [496]:
#更新某cube的rule
def update_rules(tm1:TM1Service,rules_str:str,cube:str):
    try:
        new_rules=Rules(rules_str)
        cube=tm1.cubes.get(cube)
        cube.rules=new_rules
        tm1.cubes.update(cube)
    except Exception as e:
        #写入pa日志报错信息
        tm1.server.write_to_message_log('WARN',cube+'规则更新失败')

In [497]:
#新建cellsecurity控制cube（如不存在） 
def create_cell_security_cube(tm1:TM1Service,cube:str):
        security_cube='}CellSecurity_'+cube
        dim_list=tm1.cubes.get_dimension_names(cube)   
        if tm1.cubes.exists(cube_name=security_cube)==False:
            p=''
            for i in range(len(dim_list)):
                p=p+'1:'
            p=p[:-1] 
            tm1.processes.execute_ti_code(["CellSecurityCubeCreate('"+cube+"','"+p+"');"])   

In [498]:
#根据某维度，某个汇总节点创建下属所有的数据权限组,并修改相关cube的cellsecurity
def create_groups_and_update_securityrules(tm1:TM1Service,dim:str,element:str,group_prefix:str,cube:str):
    #创建cellsecurity_cube,如果不存在
    create_cell_security_cube(tm1,cube)
    security_cube='}CellSecurity_'+cube
    desc=descendants(tm1,dim,element)
    #同时生成rule的列表
    new_rule_list=[]
    for ele in desc:
        element_exist_judge=tm1.elements.exists(dimension_name='}Groups',hierarchy_name='}Groups',element_name=group_prefix+':'+ele.name)
        if element_exist_judge==False:
            #更新}group权限组
            try:
                tm1.security.create_group(group_prefix+':'+ele.name)
            except Exception as e:
                print('报错如下：'+str(e)+'  结束')
                #写入pa日志报错信息
                tm1.server.write_to_message_log('WARN','更新}Group的'+dim+'权限的组失败:'+group_prefix+':'+ele.name)
            new_rule_list.append("['"+group_prefix+":"+ele.name+"']"+"=S:"+"IF((ELISANC("+"'"+dim+"'"+","+"'"+ele.name+"'"+","+"!"+dim+")"+"<>0 % "+"!"+dim+"@="+"'"+ele.name+"'"+")"+","+\
            "'READ',CONTINUE)")
    #现存rule的获取
    existing_rule_list=rules(tm1,security_cube)
    #整合新老rules
    pre_updating_rule_list=existing_rule_list+['#以下规则为系统自动写入']+new_rule_list
    pre_updating_rule_list.insert(0,'SKIPCHECK')
    pre_updating_rule_list.append('FEEDERS')
    pre_updating_rule_str=handle_rule_list(pre_updating_rule_list)
    #更新rules
    update_rules(tm1,pre_updating_rule_str,security_cube)

In [346]:
#建立连接
def setup(address:str,port:int,ssl:bool,user:str,password:str):
    tm1=TM1Service(address=address,port=port, ssl=ssl,user=user,password=password)
    return tm1

In [347]:
#登出
def logout(tm1:TM1Service):
    tm1.logout()

In [500]:
#测试建立D8在}group中
logging.basicConfig(level=logging.INFO,format='%(asctime)s -%(name)s -%(levelname)s - %(message)s')
if __name__=='__main__':
#     tm1=setup(address='localhost',port=30059, ssl=False,user='neil',password='123')
#     try:
#         create_groups_and_update_securityrules(tm1,'Account PL','PL100 - 三、营业利润','D8 Security','D8 Profit Loss')
#     except Exception as e:
#         logging.error(str(e))
#     finally:
#         logout(tm1)
    create_groups_and_update_securityrules(tm1,'Account PL','PL100 - 三、营业利润','D8 Security','D8 Profit Loss')

In [ ]:
tm1.cubes.

In [357]:
logging.basicConfig(level=logging.INFO,format='%(asctime)s -%(name)s -%(levelname)s - %(message)s')

In [358]:
logging.error('err')

2024-04-17 11:01:52,711 ERROR:err


In [161]:
'a' in ['a','b','c']

True

In [173]:
a="a"+\
"b"

SyntaxError: invalid syntax (<ipython-input-173-5634fa177f0c>, line 1)

In [200]:
a=[]
a.append("a")

In [201]:
a

['a']

In [176]:
b=['b']

In [177]:
a+b

['a', 'b']

In [492]:
 with TM1Service(address='localhost',port='30059', ssl='False',user='neil',password='123') as tm1:
#     hier=tm1.hierarchies.get('Account PL',hierarchy_name='Account PL')
#     desc=hier.get_descendants(element_name='PL100 - 三、营业利润',recursive=True)
#     ele=tm1.elements.get('Account PL','Account PL','PL100 - 三、营业利润')
#     desc=list(desc)
#     desc.append(ele)
#     cube=tm1.cubes.get('}CellSecurity_D8 Profit Loss')
#     rules=cube.rules
#     desc=set(desc)
#     tm1.elements.get()
    def rules(tm1:TM1Service,cube:str):
        cube=tm1.cubes.get(cube)
        rules=cube.rules
        if rules is None:
            rule_list=[]
        else:   
            rule_list=rules.rule_statements
            #忽略skipcheck
            if str.lower(rule_list[0])=='skipcheck':
                rule_list.pop(0)
            #忽略feeders
            if str.lower(rule_list[-1])=='feeders':
                rule_list.pop(-1)
        return rule_list
    r=rules(tm1,'}CellSecurity_D8 Profit Loss')
            

In [493]:
r

[]

In [489]:
desc

[Element:{"Name": "PL113 - \u5176\u4ed6\u6536\u76ca", "Type": "Numeric"},
 Element:{"Name": "PL114a - \u5176\u4ed6\u6295\u8d44\u635f\u76ca", "Type": "Numeric"},
 Element:{"Name": "PL108a - \u5176\u4ed6\u5229\u606f\u6536\u5165", "Type": "Numeric"},
 Element:{"Name": "PL103 - \u5176\u4e2d\uff1a\u8425\u4e1a\u6210\u672c", "Type": "Numeric"},
 Element:{"Name": "PL105 - \u9500\u552e\u8d39\u7528", "Type": "Numeric"},
 Element:{"Name": "PL108 - \u8d22\u52a1\u8d39\u7528", "Type": "Consolidated"},
 Element:{"Name": "PL116 - \u4ee5\u644a\u4f59\u6210\u672c\u8ba1\u91cf\u7684\u91d1\u878d\u8d44\u4ea7\u7ec8\u6b62\u786e\u8ba4\u6536\u76ca", "Type": "Numeric"},
 Element:{"Name": "PL111 - \u63d0\u53d6\u5408\u540c\u8d23\u4efb\u51c6\u5907\u91d1\u51c0\u989d", "Type": "Numeric"},
 Element:{"Name": "PL110 - \u5176\u4e2d\uff1a\u6c47\u5151\u635f\u76ca", "Type": "Numeric"},
 Element:{"Name": "PL102 - \u4e8c\u3001\u8425\u4e1a\u603b\u6210\u672c", "Type": "Consolidated"},
 Element:{"Name": "PL104 - \u7a0e\u91d1\u53c

In [471]:
for e in desc:
    print(e.name)

PL113 - 其他收益
PL114a - 其他投资损益
PL108a - 其他利息收入
PL103 - 其中：营业成本
PL105 - 销售费用
PL108 - 财务费用
PL116 - 以摊余成本计量的金融资产终止确认收益
PL111 - 提取合同责任准备金净额
PL110 - 其中：汇兑损益
PL102 - 二、营业总成本
PL104 - 税金及附加
PL107 - 研发费用
PL114 - 投资损益
PL111111
PL101 - 一、营业总收入
PL109a - 其中：利息费用
PL120 - 资产处置收益
PL106 - 管理费用
PL117 - 公允价值变动收益
PL119 - 资产减值损失
PL115 - 其中：对联营企业和合营企业投资收益
PL118 - 信用减值损失
PL109b - 其中：利息收入
PL100 - 三、营业利润


In [426]:
ele.from_dict

<function TM1py.Objects.Element.Element.from_dict(element_as_dict: Dict) -> 'Element'>

In [193]:
existing_rule_list

["['FUNCTIONAL SECURITY：CASHIER']=S:STET"]

In [190]:
str.lower('SKI')

'ski'

In [406]:
for e in list(desc):
    e.name
e.name    

'PL109b - 其中：利息收入'

In [399]:
ele

Element:{"Name": "PL100 - \u4e09\u3001\u8425\u4e1a\u5229\u6da6", "Type": "Consolidated"}

In [265]:
try:
    re=10/0
except Exception as e:
    #print(str(e))
    str(e)

In [317]:
'AAAAA'.encode()

b'AAAAA'

In [325]:
str(['a'])

"['a']"

In [483]:
str.lower('abv')

'abv'